## Importing data from Postgresql to elastic, where a dashboard will use the data to display cool stuff

In [1]:
import datetime as dt
from datetime import timedelta, date

from airflow import DAG
#from airflow.operators.bash_operator import BashOperator
#from airflow.operators.python_operator import PythonOperator

import pandas as pd
import psycopg2 as db
from elasticsearch import Elasticsearch
import urllib3
urllib3.disable_warnings()

    Query data from postgres from today only

In [6]:
con = "dbname = 'postgres' host='192.168.1.48' user='postgres' password='SliceOfLyfe69'"
connection = db.connect(con)
d = date.today()
#t = dt.timedelta(days=1)
#a = d - t
df=pd.read_sql(f"select * from scf where date = '{d}'",connection)

C:\Users\antng\AppData\Local\Temp\ipykernel_19264\3352194760.py:6 UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

In [7]:
df.head()

,id,summary,description,rating,full_address,date,html_url,zipcode,coordinate
0,13718508,Park Lighting,Black light pole is loose and wobbles side to ...,2,"Milpitas CA, USA",2022-11-19,https://seeclickfix.com/issues/13718508,0,"{""lat"":37.4387707321218,""lon"":-121.883577365609}"
1,13718464,Graffiti Removal,graffiti 16315 condit south and east sides of ...,2,"16301-16483 Condit Rd Morgan Hill, CA, 95037, USA",2022-11-19,https://seeclickfix.com/issues/13718464,95037,"{""lat"":37.1242523729142,""lon"":-121.626974723278}"
2,13717668,Vandalism / Graffitti,My vehicle was damaged by a premediated group ...,3,"190 Ranch Dr Milpitas, California, 95035",2022-11-19,https://seeclickfix.com/issues/13717668,95035,"{""lat"":37.4283980213903,""lon"":-121.919815019667}"
3,13717642,FM - Indoor Construction,Padding coming off this basketball hoop in eas...,2,"171 W Edmundson Ave Morgan Hill CA 95037, Unit...",2022-11-19,https://seeclickfix.com/issues/13717642,95037,"{""lat"":37.1132413,""lon"":-121.6457796}"
4,13717574,Traffic or Street Sign Issue,Parents frequently are ignore the sign for li...,2,"17785 Vista Ave Monte Sereno, CA, 95030, USA",2022-11-19,https://seeclickfix.com/issues/13717574,95030,"{""lat"":37.2392884330469,""lon"":-121.981868465574}"


    Save to csv file

In [8]:
df.to_csv('postgredata.csv')

    Interact with elastic

In [17]:
es = Elasticsearch(
        ['https://192.168.1.41:9200'],
        http_auth=('elastic', 'n_lYRaZWtMKXbwi_voyl'),
        verify_certs=False
    )

C:\Users\antng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\elasticsearch\_sync\client\__init__.py:395 SecurityWarning: Connecting to 'https://192.168.1.41:9200' using TLS with verify_certs=False is insecure

C:\Users\antng\AppData\Local\Temp\ipykernel_6172\1190163643.py:1 DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead

In [21]:
df=pd.read_csv('postgredata.csv')
for i,r in df.iterrows():
    doc = r.to_json()
res = es.index(index="frompostgresq",id=i,body=doc)
print(res)

C:\Users\antng\AppData\Local\Temp\ipykernel_6172\1681194311.py:4 DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information

[2022-11-17 18:25:13,226] {_transport.py:336} INFO - PUT https://192.168.1.41:9200/frompostgresq/_doc/8 [status:201 duration:0.375s]
{'_index': 'frompostgresq', '_id': '8', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


In [25]:
try:
    df=pd.read_csv('postgredata.csv')
except:
    print("IOError OR Pandas Error")
for i,r in df.iterrows():
    doc = r.to_json()
    print(r['id'])

13703524
13703017
13702440
13702351
13702192
13701906
13701701
13699710
13699614


In [28]:
try:
    df=pd.read_csv('postgredata.csv')
except:
    print("IOError OR Pandas Error")
for i,r in df.iterrows():
    doc = r.to_json()
    res = es.index(index="seeclickfix",id=r['id'],body=doc)
    print(res)

C:\Users\antng\AppData\Local\Temp\ipykernel_6172\1035608132.py:7 DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information

[2022-11-17 18:56:11,174] {_transport.py:336} INFO - PUT https://192.168.1.41:9200/seeclickfix/_doc/13703524 [status:201 duration:0.377s]
{'_index': 'seeclickfix', '_id': '13703524', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
[2022-11-17 18:56:11,204] {_transport.py:336} INFO - PUT https://192.168.1.41:9200/seeclickfix/_doc/13703017 [status:201 duration:0.018s]
{'_index': 'seeclickfix', '_id': '13703017', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
[2022-11-17 18:56:11,235] {_transport.py:336} INFO - PUT https://192.168.1.41:9200/seeclickfix/_doc/13702440 [status:201 duration:0.019s]
{'_index': 'seeclickfix', '_id': '13702440', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
[2022-11-17 18:56:11,265] {_transport.py:336} INFO - PUT https://192.168.1.4

In [29]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    ['https://192.168.1.41:9200'],
    basic_auth=('elastic', 'n_lYRaZWtMKXbwi_voyl'),
    verify_certs=False
)
result = es.search(index='frompostgresq',size=10)

C:\Users\antng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\elasticsearch\_sync\client\__init__.py:395 SecurityWarning: Connecting to 'https://192.168.1.41:9200' using TLS with verify_certs=False is insecure

[2022-11-17 19:00:33,048] {_transport.py:336} INFO - POST https://192.168.1.41:9200/frompostgresq/_search [status:200 duration:0.122s]


In [31]:
print(result['hits']['hits'])

[{'_index': 'frompostgresq', '_id': '999', '_score': 1.0, '_source': {'Unnamed: 0': 999, 'name': 'Judy Gray', 'city': 'Pattersonburgh'}}]


In [45]:
def queryPostgresql():
    con = "dbname = 'postgres' host='192.168.1.41' user='postgres' password='SliceOfLyfe69'"
    try:
        connection = db.connect(con)
    except:
        print("Error Connecting to Postgres Database")
    try:
        df=pd.read_sql(f"select * from seeclickfix where date = '2022-11-15'",connection)
    except:
        print("Error querying data from postgres OR pandas related Error")
    print(df.head())
    df.to_csv('postgredata.csv')
    print("------Data Received Successfully from Postgres------")
def insertElasticsearch():    
    es = Elasticsearch(
        ['https://192.168.1.41:9200'],
        http_auth=('elastic', 'n_lYRaZWtMKXbwi_voyl'),
        verify_certs=False
    )
    df=pd.read_csv('postgredata.csv')
    for i,r in df.iterrows():
        doc = r.to_json()
        print(doc)
        res = es.index(index="seeclickfix",id=r['id'],body=doc)
        print(res)
    print("----Data Saved to Elastic!----")
    os.remove('postgredata.csv')

In [46]:
queryPostgresql()
insertElasticsearch()

C:\Users\antng\AppData\Local\Temp\ipykernel_6172\3641198477.py:8 UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

         id                                         summary  \
0  13703524                                  Shopping Carts   
1  13703017                           Vandalism / Graffitti   
2  13702440  Weed/Brush Abatement and Landscape Maintenance   
3  13702351            Street Signs, Striping, and Markings   
4  13702192                        Debris / Illegal Dumping   

                                         description  rating        lat  \
0  Found a shopping cart on the sidewalk across f...       1  37.413721   
1                            Graffiti on Utility box       2  37.423705   
2  Hedges along street are not getting water and ...       2  37.427242   
3  New name plate for New PW Director Christian D...       2  37.448344   
4                       Reports of debris in roadway       2  37.445960   

         long                                            address        date  
0 -121.907905     240-268 W Capitol Ave Milpitas, CA, 95035, USA  2022-11-17  
1 -121.86936

C:\Users\antng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\elasticsearch\_sync\client\__init__.py:395 SecurityWarning: Connecting to 'https://192.168.1.41:9200' using TLS with verify_certs=False is insecure

C:\Users\antng\AppData\Local\Temp\ipykernel_6172\3641198477.py:15 DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead

{"Unnamed: 0":0,"id":13703524,"summary":"Shopping Carts","description":"Found a shopping cart on the sidewalk across from 267\/275 W Capitol Ave. please help to take care of it. ","rating":1,"lat":37.4137209765,"long":-121.9079054681,"address":"240-268 W Capitol Ave Milpitas, CA, 95035, USA","date":"2022-11-17"}


C:\Users\antng\AppData\Local\Temp\ipykernel_6172\3641198477.py:24 DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information

[2022-11-17 19:44:28,365] {_transport.py:336} INFO - PUT https://192.168.1.41:9200/seeclickfix/_doc/13703524 [status:201 duration:0.149s]
{'_index': 'seeclickfix', '_id': '13703524', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 6, '_primary_term': 1}
{"Unnamed: 0":1,"id":13703017,"summary":"Vandalism \/ Graffitti","description":"Graffiti on Utility box","rating":2,"lat":37.4237053577,"long":-121.8693649728,"address":"Westridge Dr Milpitas, California, 95035","date":"2022-11-17"}
[2022-11-17 19:44:28,400] {_transport.py:336} INFO - PUT https://192.168.1.41:9200/seeclickfix/_doc/13703017 [status:201 duration:0.019s]
{'_index': 'seeclickfix', '_id': '13703017', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7, '_primary_term': 1}
{"Unnamed: 0":2,"id":13702440,"summary":"Weed\/Brush Abatement and Landscape Maintenance","description":"Hedges along street are not getting water and 